In [1]:
from sagemaker.huggingface.model import HuggingFaceModel
import sagemaker
from sagemaker import get_execution_role

In [2]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = f"az-ade-{sess.account_id()}"

In [12]:
%store -r training_job_name
training_job_name

'az-ade-training-2021-10-05-11-20-24-189'

In [3]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data="s3://az-ade-905847418383/training_output/huggingface-pytorch-training-2021-10-05-10-04-06-248/output/model.tar.gz",  # path to your trained sagemaker model
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.6", # transformers version used
   pytorch_version="1.7", # pytorch version used
   py_version="py36", # python version of the DLC
)

In [4]:
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.p3.2xlarge"
)

-------------------!

In [5]:
import pandas as pd

In [6]:
from datasets import load_from_disk
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem(session=sess)

dataset_test = load_from_disk('s3://az-ade-905847418383/processing_output/test_data/',fs=s3)
df_test = pd.DataFrame(dataset_test)

In [7]:
df_test.head(10)

,__index_level_0__,attention_mask,input_ids,label,text
0,4240,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2057, 3189, 2006, 1037, 5179, 1011, 2095...",1,We report on a 56-year-old female who exhibite...
1,10746,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2053, 3276, 2090, 26445, 23722, 18098, 5...",0,No relationship between amisulpride and rhabdo...
2,15974,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 17758, 13749, 20023, 2389, 8040, 3217, 3...",0,Bilateral inguinal scrofuloderma during steroi...
3,20248,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 25732, 5038, 1997, 2023, 8715, 1998, 574...",0,Prompt recognition of this syndrome and tempor...
4,13658,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[101, 20934, 8458, 24090, 15530, 2206, 22575, ...",0,Buphthalmos following systemic steroid treatment.
5,5732,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2028, 3287, 5776, 2007, 17758, 1040, 416...",0,One male patient with bilateral DME was treate...
6,18539,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 4281, 1024, 3078, 1051, 10755, 2937, 104...",0,BACKGROUND: Primary ovarian lymphoma is an ext...
7,9652,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2348, 2023, 7242, 2038, 1037, 2200, 2659...",0,Although this therapy has a very low toxicity ...
8,18125,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 5001, 10464, 20389, 2100, 3936, 2009, 20...",0,Laparotomy revealed it to be unresectable.
9,2259,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 4475, 1998, 4725, 1024, 2057, 2556, 2048...",1,MATERIALS AND METHODS: We present two cases of...


In [8]:
ade_input= {"inputs": df_test.iloc[0]['text']}
predictor.predict(ade_input)

[{'label': 'LABEL_1', 'score': 0.7323219776153564}]

In [9]:
ade_input= {"inputs": df_test.iloc[8]['text']}
predictor.predict(ade_input)

[{'label': 'LABEL_0', 'score': 0.9939584136009216}]

In [10]:
ade_input= {"inputs": "I got a rash from taking aspirin"}
predictor.predict(ade_input)

[{'label': 'LABEL_1', 'score': 0.7767961621284485}]

In [11]:
ade_input= {"inputs": "I watched football and got really excited"}
predictor.predict(ade_input)

[{'label': 'LABEL_0', 'score': 0.9457708597183228}]